I have close to 180k entries.  
If I take 2 seconds per annotation I can annotate 18k records in 10 hours.

In [1]:
import pandas as pd
import tkinter as tk
import os.path
from tkinter import Tk, Button, Frame, Text, messagebox
import xlsxwriter # cannot modify excel files
from openpyxl import load_workbook # can modify excel files
import re
from Dataframe_Cleaner import Cleaner

In [2]:
# Creates workbook with non annotated data
def create_new_annotation_excel_file():
    submissions = pd.read_pickle("./data/submissions.pkl")
    cl = Cleaner(submissions)
    submissions = cl.missing_content_flag()
    workbook = xlsxwriter.Workbook('./data/annotation.xlsx')
    worksheet = workbook.add_worksheet()

    for i in range(0, len(submissions["title"]), 10):
        # row, column, item
        worksheet.write(int(i/10), 0, i)
        worksheet.write(int(i/10), 1, submissions["title"][i])
        if submissions["missing_content"][i] == True:
            worksheet.write(int(i/10), 2, " ")
        else:
            worksheet.write(int(i/10), 2, str(re.sub(r'\n',' ', submissions["selftext"][i][:100])))
        worksheet.write(int(i/10), 3, "?")

    workbook.close()

    wb = load_workbook(filename = "./data/annotation.xlsx")
    ws = wb.active

    ws["E2"] = "Number of annotations:"
    ws["E3"] = 0

    wb.save(filename = "./data/annotation.xlsx")

if os.path.isfile('./data/annotation.xlsx'):
    print("File already exists!")
else:
    create_new_annotation_excel_file()


File already exists!


In [3]:
def load_annotation():
    wb = load_workbook(filename = "./data/annotation.xlsx")
    ws = wb.active
    return ws, wb

ws, wb = load_annotation()

def get_annotation_count():
    i = ws["E3"].value
    return i

def increment_annotation_count():
    i = get_annotation_count()
    i = i + 1
    ws["E3"] = i

def save_to_excel():
    wb.save(filename = "./data/annotation.xlsx")

def count_annotations():
    x = get_annotation_count()
    m = len(ws['A'])

    return str(x) + " / " + str(m)

In [9]:
def bullish_func_button():
    increment_annotation_count()
    row = get_annotation_count()
    cell = "D" + str(row)
    ws[cell] = "bullish"

def bearish_func_button():
    increment_annotation_count()
    row = get_annotation_count()
    cell = "D" + str(row)
    ws[cell] = "bearish"

def neutral_func_button():
    increment_annotation_count()
    row = get_annotation_count()
    cell = "D" + str(row)
    ws[cell] = "neutral"

def bullish_func(self):
    bullish_func_button()
    gui.title(count_annotations())
    update_label()
    update_content()

def bearish_func(self):
    bearish_func_button()
    gui.title(count_annotations())
    update_label()
    update_content()

def neutral_func(self):
    neutral_func_button()
    gui.title(count_annotations())
    update_label()
    update_content()

def on_saving():
    save_to_excel()
    return ""

def update_label():
    row = get_annotation_count() + 1
    cell = "B" + str(row)
    label.config(text=ws[cell].value)

def update_content():
    row = get_annotation_count() + 1
    cell = "C" + str(row)
    content.config(text=ws[cell].value)

gui = tk.Tk()

BUTTON_HEIGHT = 3
BUTTON_WIDTH = 10

gui.config(bg = "#222222")
gui.geometry("500x500")
gui.title(count_annotations())

# Bullish button
bullish = Button(gui, text = "bullish <a>", command = bullish_func_button)
gui.bind('a', bullish_func)
bullish.config(height = BUTTON_HEIGHT, width = BUTTON_WIDTH)
bullish.place(x=5,y=440)

# Bearish button
bearish = Button(gui, text = "bearish <b>", command = bearish_func_button)
bearish.config(height = BUTTON_HEIGHT, width = BUTTON_WIDTH)
gui.bind('b', bearish_func)
bearish.place(x=415,y=440)

# Neutral button
neutral = Button(gui, text = "Neutral < >", command = neutral_func_button)
neutral.config(height = BUTTON_HEIGHT, width = BUTTON_WIDTH)
gui.bind('<space>', neutral_func)
neutral.place(x=205,y=440)

# Title    
row = get_annotation_count() + 1
cell = "B" + str(row)
label = tk.Label(gui, justify = tk.CENTER, pady = 10, wraplengt=400, font = ("Arial", 15), text = ws[cell].value)
label.pack(pady = 50)

# Content
row = get_annotation_count() + 1
cell = "C" + str(row)
content = tk.Label(gui, justify = tk.CENTER, pady = 2, wraplengt=400, font = ("Arial", 15), text = ws[cell].value)
content.pack(pady = 50)
print(cell)

# Menu
menubar = tk.Menu(gui)
filemenu = tk.Menu(menubar, tearoff=0)
filemenu.add_command(label="Save", command=on_saving)
menubar.add_cascade(label="File", menu=filemenu)
gui.config(menu=menubar)

gui.mainloop()

C16501
